In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
# from matplotlib import pyplot as plt
%matplotlib inline
plt.style.use('ggplot')
import plotly.graph_objects as go
from plotly import offline as pyoff
RANDOM_STATE = 42
from datetime import date
import calendar
import datetime as dt
from datetime import datetime

In [2]:
df = pd.read_csv('df_banc_rev.csv')

In [3]:
###Para obtener el listado de los 10000 mejores clientes, a quienes enviar el email, vamos a seguir el criterio de
###los que han hecho mas operaciones de compra con nosotros; preferimos este criterio, en lugar del revenue, porque
##teniendo en cuenta el alto nivel de abandono (archivo Churn2) que tenemos, es mas facil fidelizar y a su vez, con un
##porcentaje de interes frente a nuestra mailingm aquel cliente,
## que nos compra mas veces, y ha experimentado  nuestro servicio de atencio al cliente,
###que el que solo lo hace una vez, y nos pueda dar más revenue.

In [48]:
df_top = df.groupby(by=['pk_cid','entry_date'],as_index=False)["unidades_vendidas_total"].count()

In [49]:
df_top

,pk_cid,entry_date,unidades_vendidas_total
0,15891,2018-07-28,2
1,16063,2018-11-19,7
2,16203,2018-12-23,6
3,16502,2018-09-30,9
4,17457,2017-09-16,17
...,...,...,...
454901,1553685,2019-05-31,1
454902,1553686,2019-05-31,1
454903,1553687,2019-05-31,1
454904,1553688,2019-05-31,1


In [50]:
df_top = df_top[df_top['unidades_vendidas_total']==17]

In [53]:
df_top.describe()

,pk_cid,unidades_vendidas_total
count,2.328910e+05,232891.0
mean,1.178941e+06,17.0
std,1.261654e+05,0.0
min,1.745700e+04,17.0
25%,1.086818e+06,17.0
50%,1.180961e+06,17.0
75%,1.280713e+06,17.0
max,1.379128e+06,17.0


In [55]:
NOW = dt.datetime(2019,5,28)

In [58]:
df['entry_date'] = pd.to_datetime(df['entry_date'])

In [59]:
df['pk_partition'] = pd.to_datetime(df['pk_partition'])

In [63]:
df_17m = df[(df.entry_date.dt.date < date(2019,5,28)) & (df.entry_date.dt.date >= date(2018,1,1))].reset_index(drop=True)

In [64]:
df_frequency = df_17m.groupby('pk_cid').entry_date.count().reset_index()
df_frequency.columns = ['pk_cid','Frequency']

In [65]:
df = pd.merge(df,df_frequency,on='pk_cid',how='right')

In [66]:
df.head(20).T

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
Unnamed: 0,2168122,1479563,5134317,2962973,4028169,5523397,4876040,3628236,4480637,5134321,5523396,3628238,4028170,4876039,4480636,4876038,3628237,2171490,5523395,5134320
pk_cid,15891,15891,16063,16063,16063,16063,16063,16063,16063,16203,16203,16203,16203,16203,16203,16502,16502,16502,16502,16502
pk_partition,2018-08-28 00:00:00,2018-07-28 00:00:00,2019-04-28 00:00:00,2018-11-28 00:00:00,2019-01-28 00:00:00,2019-05-28 00:00:00,2019-03-28 00:00:00,2018-12-28 00:00:00,2019-02-28 00:00:00,2019-04-28 00:00:00,2019-05-28 00:00:00,2018-12-28 00:00:00,2019-01-28 00:00:00,2019-03-28 00:00:00,2019-02-28 00:00:00,2019-03-28 00:00:00,2018-12-28 00:00:00,2018-09-28 00:00:00,2019-05-28 00:00:00,2019-04-28 00:00:00
entry_date,2018-07-28 00:00:00,2018-07-28 00:00:00,2018-11-19 00:00:00,2018-11-19 00:00:00,2018-11-19 00:00:00,2018-11-19 00:00:00,2018-11-19 00:00:00,2018-11-19 00:00:00,2018-11-19 00:00:00,2018-12-23 00:00:00,2018-12-23 00:00:00,2018-12-23 00:00:00,2018-12-23 00:00:00,2018-12-23 00:00:00,2018-12-23 00:00:00,2018-09-30 00:00:00,2018-09-30 00:00:00,2018-09-30 00:00:00,2018-09-30 00:00:00,2018-09-30 00:00:00
active_customer,0,1,1,1,1,0,0,1,1,1,1,0,1,1,1,1,1,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
revenue_payroll,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
revenue_payroll_account,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
revenue_total,0,10,0,0,0,0,0,0,0,10,10,0,10,10,10,20,10,0,20,20
unidades_vendidas_total,0,1,0,0,0,0,0,0,0,1,1,0,1,1,1,2,1,0,2,2


In [68]:
rfmTable = df.groupby('pk_cid').agg({'pk_partition': lambda x: (NOW - x.max()).days,'Frequency': lambda x: len(x), 'revenue_total': lambda x: x.sum()})
rfmTable['entry_date'] = rfmTable['pk_partition'].astype(int)
rfmTable.rename(columns={'entry_date': 'recency', 
                         'Frequency': 'frequency',
                         'revenue_total': 'monetary_value'}, inplace=True)

In [69]:
rfmTable

,pk_partition,frequency,monetary_value,recency
pk_cid,,,,
15891,273,2,10.0,273
16063,0,7,0.0,0
16203,0,6,50.0,0
16502,0,9,130.0,0
17799,0,17,170.0,0
...,...,...,...,...
1553111,0,1,0.0,0
1553112,0,1,0.0,0
1553113,0,1,0.0,0


In [74]:
first_customer=df[df['pk_cid']==16502]

In [75]:
first_customer.T

,15,16,17,18,19,20,21,22,23
Unnamed: 0,4876038,3628237,2171490,5523395,5134320,4480638,4028330,2962972,2943633
pk_cid,16502,16502,16502,16502,16502,16502,16502,16502,16502
pk_partition,2019-03-28 00:00:00,2018-12-28 00:00:00,2018-09-28 00:00:00,2019-05-28 00:00:00,2019-04-28 00:00:00,2019-02-28 00:00:00,2019-01-28 00:00:00,2018-11-28 00:00:00,2018-10-28 00:00:00
entry_date,2018-09-30 00:00:00,2018-09-30 00:00:00,2018-09-30 00:00:00,2018-09-30 00:00:00,2018-09-30 00:00:00,2018-09-30 00:00:00,2018-09-30 00:00:00,2018-09-30 00:00:00,2018-09-30 00:00:00
active_customer,1,1,0,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...
revenue_payroll,0,0,0,0,0,0,0,0,0
revenue_payroll_account,0,0,0,0,0,0,0,0,0
revenue_total,20,10,0,20,20,20,20,10,10
unidades_vendidas_total,2,1,0,2,2,2,2,1,1


In [76]:
quantiles = rfmTable.quantile(q=[0.25,0.5,0.75])
quantiles = quantiles.to_dict()

In [77]:
segmented_rfm = rfmTable

In [78]:
###The lowest recency, highest frequency and monetary amounts are our best customers.

In [79]:
def RScore(x,p,d):
    if x <= d[p][0.25]:
        return 1
    elif x <= d[p][0.50]:
        return 2
    elif x <= d[p][0.75]: 
        return 3
    else:
        return 4

In [80]:
def FMScore(x,p,d):
    if x <= d[p][0.25]:
        return 4
    elif x <= d[p][0.50]:
        return 3
    elif x <= d[p][0.75]: 
        return 2
    else:
        return 1

In [81]:
segmented_rfm['r_quartile'] = segmented_rfm['recency'].apply(RScore, args=('recency',quantiles,))
segmented_rfm['f_quartile'] = segmented_rfm['frequency'].apply(FMScore, args=('frequency',quantiles,))
segmented_rfm['m_quartile'] = segmented_rfm['monetary_value'].apply(FMScore, args=('monetary_value',quantiles,))
segmented_rfm.head()

,pk_partition,frequency,monetary_value,recency,r_quartile,f_quartile,m_quartile
pk_cid,,,,,,,
15891,273,2,10.0,273,4,4,3
16063,0,7,0.0,0,1,3,4
16203,0,6,50.0,0,1,4,3
16502,0,9,130.0,0,1,2,1
17799,0,17,170.0,0,1,1,1


In [84]:
segmented_rfm['RFMScore'] = segmented_rfm.r_quartile.map(str) + segmented_rfm.f_quartile.map(str) + segmented_rfm.m_quartile.map(str)


In [85]:
segmented_rfm.head()

,pk_partition,frequency,monetary_value,recency,r_quartile,f_quartile,m_quartile,RFMScore
pk_cid,,,,,,,,
15891,273,2,10.0,273,4,4,3,443
16063,0,7,0.0,0,1,3,4,134
16203,0,6,50.0,0,1,4,3,143
16502,0,9,130.0,0,1,2,1,121
17799,0,17,170.0,0,1,1,1,111


In [86]:
##It is obvious that the first customer is not our best customer at all.

In [87]:
segmented_rfm[segmented_rfm['RFMScore']=='111'].sort_values('monetary_value', ascending=False).head(10000)

,pk_partition,frequency,monetary_value,recency,r_quartile,f_quartile,m_quartile,RFMScore
pk_cid,,,,,,,,
1374819,0,17,4290.0,0,1,1,1,111
1392320,0,14,3190.0,0,1,1,1,111
1377496,0,17,2880.0,0,1,1,1,111
1374123,0,17,2850.0,0,1,1,1,111
1374403,0,17,2840.0,0,1,1,1,111
...,...,...,...,...,...,...,...,...
1375028,0,17,260.0,0,1,1,1,111
1385315,0,15,260.0,0,1,1,1,111
1259466,0,15,260.0,0,1,1,1,111
